In [1]:
import pandas as pd

In [3]:
!pip3 install openpyxl

Defaulting to user installation because normal site-packages is not writeable
     |████████████████████████████████| 250 kB 5.3 MB/s eta 0:00:01
You should consider upgrading via the '/Library/Developer/CommandLineTools/usr/bin/python3 -m pip install --upgrade pip' command.


In [2]:
train_raw = pd.read_excel('Data_Train.xlsx')
# Drop rows with missing values
train_raw.dropna(inplace=True)
train_raw.reset_index(drop=True, inplace=True)

test_raw = pd.read_excel('Test_set.xlsx')
# Drop rows with missing values
test_raw.dropna(inplace=True)
test_raw.reset_index(drop=True, inplace=True)

In [3]:
train_raw.head()

,Airline,Date_of_Journey,Source,Destination,Route,Dep_Time,Arrival_Time,Duration,Total_Stops,Additional_Info,Price
0,IndiGo,24/03/2019,Banglore,New Delhi,BLR → DEL,22:20,01:10 22 Mar,2h 50m,non-stop,No info,3897
1,Air India,1/05/2019,Kolkata,Banglore,CCU → IXR → BBI → BLR,05:50,13:15,7h 25m,2 stops,No info,7662
2,Jet Airways,9/06/2019,Delhi,Cochin,DEL → LKO → BOM → COK,09:25,04:25 10 Jun,19h,2 stops,No info,13882
3,IndiGo,12/05/2019,Kolkata,Banglore,CCU → NAG → BLR,18:05,23:30,5h 25m,1 stop,No info,6218
4,IndiGo,01/03/2019,Banglore,New Delhi,BLR → NAG → DEL,16:50,21:35,4h 45m,1 stop,No info,13302


In [4]:
# Initialize empty DF to store processed data
train = pd.DataFrame()

In [5]:
# Convert Date_of_Journey to datetime
train['date_of_journey'] = pd.to_datetime(train_raw['Date_of_Journey'], dayfirst=True)

# Extract day of week, month and year from Date_of_Journey
train['day_of_week'] = train['date_of_journey'].dt.dayofweek
train['month'] = train['date_of_journey'].dt.month
train['year'] = train['date_of_journey'].dt.year

In [6]:
# Convert Total_Stops to numeric
def extract_stops(stops_str):
    if isinstance(stops_str, str):
        if 'non-stop' in stops_str:
            return 0
        elif 'stop' in stops_str:
            return int(stops_str.split()[0])
    return 0

train['total_stops'] = train_raw['Total_Stops'].apply(extract_stops)

In [7]:
# Extract hour of day from Dep_Time and Arrival_Time, rounded to nearest hour
def extract_hour(time_str):
    time_str_split = time_str.split(':')
    hour = int(time_str_split[0])
    minute = int(time_str_split[1].split()[0])
    if minute >= 30:
        return hour + 1
    return hour

train['dep_hour'] = train_raw['Dep_Time'].apply(extract_hour)
train['arr_hour'] = train_raw['Arrival_Time'].apply(extract_hour)

In [8]:
# Convert Duration to minutes
def duration_to_minutes(duration_str):
    duration_str_split = duration_str.split()
    if len(duration_str_split) == 2:
        return int(duration_str_split[0][:-1]) * 60 + int(duration_str_split[1][:-1])
    elif 'h' in duration_str:
        return int(duration_str_split[0][:-1]) * 60
    return int(duration_str_split[0][:-1])

train['duration'] = train_raw['Duration'].apply(duration_to_minutes)

In [9]:
# There are may rows where in source or destination places like 'New Delhi' and 'Delhi' are used interchangeably
# To fix this we will replace these names with airport codes extracted from Route column

# Extract source and destination from Route
train['source'] = train_raw['Route'].apply(lambda x: x.split('→')[0].strip())
train['destination'] = train_raw['Route'].apply(lambda x: x.split('→')[-1].strip())

In [10]:
# Calculate route complexity
train['route_complexity'] = train_raw['Route'].apply(lambda x: len(x.split('→')) - 2)

In [11]:
train_raw['Additional_Info'].value_counts()

Additional_Info
No info                         8344
In-flight meal not included     1982
No check-in baggage included     320
1 Long layover                    19
Change airports                    7
Business class                     4
No Info                            3
1 Short layover                    1
Red-eye flight                     1
2 Long layover                     1
Name: count, dtype: int64

In [12]:
# Process Additional_Info
def process_additional_info(info_str):
    has_meal = 1 if 'meal' in info_str.lower() else 0
    # invert has_meal as 1 means no meal
    has_meal != has_meal

    check_in_baggage = 1 if 'check-in baggage' in info_str.lower() else 0
    # invert check_in_baggage as 1 means no check-in baggage
    check_in_baggage != check_in_baggage

    short_layover = int(info_str.split()[0]) if 'short' in info_str.lower() else 0
    long_layover = int(info_str.split()[0]) if 'long' in info_str.lower() else 0
    
    red_eye = 1 if 'red-eye' in info_str.lower() else 0

    business_class = 1 if 'business' in info_str.lower() else 0

    change_airport = 1 if 'change' in info_str.lower() else 0

    return pd.Series({
        'has_meal': has_meal,
        'check_in_baggage': check_in_baggage,
        'short_layover': short_layover,
        'long_layover': long_layover,
        'red_eye': red_eye,
        'business_class': business_class,
        'change_airport': change_airport
    })


additional_info_df = train_raw['Additional_Info'].apply(process_additional_info)
train = pd.concat([train, additional_info_df], axis=1)

In [13]:
# Add Airline column
train['airline'] = train_raw['Airline']
# Add Price column
train['price'] = train_raw['Price']

In [14]:
train.head()

,date_of_journey,day_of_week,month,year,total_stops,dep_hour,arr_hour,duration,source,destination,route_complexity,has_meal,check_in_baggage,short_layover,long_layover,red_eye,business_class,change_airport,airline,price
0,2019-03-24,6,3,2019,0,22,1,170,BLR,DEL,0,0,0,0,0,0,0,0,IndiGo,3897
1,2019-05-01,2,5,2019,2,6,13,445,CCU,BLR,2,0,0,0,0,0,0,0,Air India,7662
2,2019-06-09,6,6,2019,2,9,4,1140,DEL,COK,2,0,0,0,0,0,0,0,Jet Airways,13882
3,2019-05-12,6,5,2019,1,18,24,325,CCU,BLR,1,0,0,0,0,0,0,0,IndiGo,6218
4,2019-03-01,4,3,2019,1,17,22,285,BLR,DEL,1,0,0,0,0,0,0,0,IndiGo,13302


In [15]:
from sklearn.model_selection import train_test_split


X = train.drop(['date_of_journey', 'price'], axis=1)
Y = train['price']

X_train, X_test, Y_train, Y_test = train_test_split(X, Y, test_size=0.2, random_state=42)

In [16]:
# Create preprocessor
from sklearn.compose import ColumnTransformer
from sklearn.discriminant_analysis import StandardScaler
from sklearn.preprocessing import OneHotEncoder

categorical_features = ['source', 'destination', 'airline']
numerical_features = ['day_of_week', 'month', 'year', 'total_stops', 'dep_hour', 'arr_hour', 'duration', 'route_complexity', 'has_meal', 'check_in_baggage', 'short_layover', 'long_layover', 'red_eye', 'business_class', 'change_airport']

preprocessor = ColumnTransformer(
    transformers=[
        ('num', StandardScaler(), numerical_features), # Standardize numerical features
        ('cat', OneHotEncoder(handle_unknown='ignore'), categorical_features) # One-hot encode categorical features
    ]
)

In [ ]:
# # Different models to try
# from sklearn.ensemble import GradientBoostingRegressor, RandomForestRegressor
# from sklearn.linear_model import LinearRegression
# from xgboost import XGBRegressor


# models = {
#     "Linear Regression": LinearRegression(),
#     "Random Forest": RandomForestRegressor(),
#     "Gradient Boosting": GradientBoostingRegressor(),
#     "XGBoost": XGBRegressor()
# }

# results = {}

In [23]:
# Different models to try
import numpy as np
from sklearn.metrics import mean_absolute_error, mean_squared_error, r2_score
from sklearn.model_selection import cross_val_score
from sklearn.neural_network import MLPRegressor
from sklearn.pipeline import Pipeline
from sklearn.ensemble import GradientBoostingRegressor, RandomForestRegressor
from sklearn.linear_model import LinearRegression
from sklearn.model_selection import GridSearchCV, RandomizedSearchCV
import xgboost as xgb
from sklearn.preprocessing import StandardScaler
from sklearn.pipeline import Pipeline
from sklearn.neural_network import MLPRegressor

# Example: Replace 'preprocessor' with your actual preprocessing pipeline
mlp_pipeline = Pipeline(steps=[
    ('preprocessor', preprocessor),  # Must include StandardScaler inside
    ('model', MLPRegressor(max_iter=2000, random_state=42))
])


# Define models and their parameter grids
models = {
    "Linear Regression": {
        "model": LinearRegression(),
        "params": {}
    },
    "Random Forest": {
        "model": RandomForestRegressor(),
        "params": {
            "model__n_estimators": [100, 200, 300],
            "model__max_depth": [10, 20, 30],
            "model__min_samples_split": [2, 5, 10],
            "model__min_samples_leaf": [1, 2, 4]
        }
    },
    "Gradient Boosting": {
        "model": GradientBoostingRegressor(),
        "params": {
            "model__n_estimators": [100, 200, 300],
            "model__learning_rate": [0.01, 0.05, 0.1],
            "model__max_depth": [3, 5, 7],
            "model__min_samples_split": [2, 5],
            "model__subsample": [0.8, 0.9, 1.0]
        }
    },
    "XGBoost": {
        "model": xgb.XGBRegressor(objective='reg:squarederror'),
        "params": {
            "model__n_estimators": [100, 200, 300],
            "model__learning_rate": [0.01, 0.05, 0.1],
            "model__max_depth": [3, 5, 7],
            "model__min_child_weight": [1, 3, 5],
            "model__subsample": [0.8, 0.9, 1.0],
            "model__colsample_bytree": [0.8, 0.9, 1.0]
        }
    },
    "MLP": {
        "model": MLPRegressor(random_state=42),
        "params": {
            'model__hidden_layer_sizes': [(50,), (50, 50), (100,), (100, 50)],
            'model__activation': ['relu', 'tanh'],
            'model__alpha': [0.0001, 0.001, 0.01],  # Regularization
            'model__learning_rate_init': [0.001, 0.01],
            'model__solver': ['adam'],
            'model__early_stopping': [True],
            'model__validation_fraction': [0.1],
            'model__tol': [1e-3, 1e-4],
            'model__max_iter': [1000, 2000]
        }

        # "params": {
        #     "model__hidden_layer_sizes": [(100,), (50, 50)],
        #     "model__activation": ["relu", "tanh"],
        #     "model__solver": ["adam"],
        #     "model__alpha": [0.0001, 0.001],
        #     "model__max_iter": [500, 1000],
        #     "model__early_stopping": [True],
        # }
    }
}

results = {}

# Training and Evaluation
for name, model_info in models.items():
    print(f'\nTraining {name}...')
    
    # Create pipeline
    pipeline = Pipeline(steps=[('preprocessor', preprocessor), 
                             ('model', model_info['model'])])
    
    # If there are parameters to tune, use GridSearchCV
    if model_info['params']:
        grid_search = GridSearchCV(
            pipeline,
            param_grid=model_info['params'],
            cv=5,
            scoring='neg_mean_squared_error',
            n_jobs=-1
        )
        grid_search.fit(X_train, Y_train)
        best_model = grid_search.best_estimator_
    else:
        # For Linear Regression, just fit the pipeline
        pipeline.fit(X_train, Y_train)
        best_model = pipeline

    # Predict on test set
    Y_pred = best_model.predict(X_test)

    # Calculate metrics
    rmse = np.sqrt(mean_squared_error(Y_test, Y_pred))
    mae = mean_absolute_error(Y_test, Y_pred)
    r2 = r2_score(Y_test, Y_pred)

    results[name] = {
        'Test RMSE': rmse,
        'Test MAE': mae,
        'Test R2': r2,
        'Model': best_model
    }

    print(f'Test RMSE: {rmse:.2f}')
    print(f'Test MAE: {mae:.2f}')
    print(f'Test R2: {r2:.2f}')
    if model_info['params']:
        print(f'Best parameters: {grid_search.best_params_}')


Training Linear Regression...
Test RMSE: 2711.87
Test MAE: 1811.25
Test R2: 0.66

Training Random Forest...
Test RMSE: 1884.53
Test MAE: 887.89
Test R2: 0.84
Best parameters: {'model__max_depth': 30, 'model__min_samples_leaf': 1, 'model__min_samples_split': 10, 'model__n_estimators': 200}

Training Gradient Boosting...
Test RMSE: 1805.66
Test MAE: 1029.26
Test R2: 0.85
Best parameters: {'model__learning_rate': 0.1, 'model__max_depth': 5, 'model__min_samples_split': 2, 'model__n_estimators': 300, 'model__subsample': 1.0}

Training XGBoost...
Test RMSE: 1806.88
Test MAE: 917.86
Test R2: 0.85
Best parameters: {'model__colsample_bytree': 0.9, 'model__learning_rate': 0.1, 'model__max_depth': 7, 'model__min_child_weight': 3, 'model__n_estimators': 300, 'model__subsample': 1.0}

Training MLP...


/Users/shifaafreensiddiqui/Library/Python/3.9/lib/python/site-packages/sklearn/neural_network/_multilayer_perceptron.py:691: ConvergenceWarning: Stochastic Optimizer: Maximum iterations (1000) reached and the optimization hasn't converged yet.
  warnings.warn(
/Users/shifaafreensiddiqui/Library/Python/3.9/lib/python/site-packages/sklearn/neural_network/_multilayer_perceptron.py:691: ConvergenceWarning: Stochastic Optimizer: Maximum iterations (1000) reached and the optimization hasn't converged yet.
  warnings.warn(
/Users/shifaafreensiddiqui/Library/Python/3.9/lib/python/site-packages/sklearn/neural_network/_multilayer_perceptron.py:691: ConvergenceWarning: Stochastic Optimizer: Maximum iterations (1000) reached and the optimization hasn't converged yet.
  warnings.warn(
/Users/shifaafreensiddiqui/Library/Python/3.9/lib/python/site-packages/sklearn/neural_network/_multilayer_perceptron.py:691: ConvergenceWarning: Stochastic Optimizer: Maximum iterations (1000) reached and the optimiza

Test RMSE: 2205.33
Test MAE: 1292.24
Test R2: 0.77
Best parameters: {'model__activation': 'relu', 'model__alpha': 0.01, 'model__early_stopping': True, 'model__hidden_layer_sizes': (100, 50), 'model__learning_rate_init': 0.01, 'model__max_iter': 1000, 'model__solver': 'adam', 'model__tol': 0.0001, 'model__validation_fraction': 0.1}


In [25]:
import numpy as np
from sklearn.metrics import mean_absolute_error, mean_squared_error, r2_score
from sklearn.model_selection import cross_val_score
from sklearn.pipeline import Pipeline

# Training and Evaluation
for name, model_info in models.items():
    estimator = model_info["model"]  # get the actual model, e.g., LinearRegression()
    # Create pipeline
    pipeline = Pipeline(steps=[('preprocessor', preprocessor), ('model', estimator)])

    # Cross-validation
    cv_scores = cross_val_score(pipeline, X_train, Y_train, cv=5, scoring='neg_mean_squared_error')
    # Calculate RMSE
    rmse_scores = np.sqrt(-cv_scores)

    # Train on full training set
    pipeline.fit(X_train, Y_train)
    # Predict on test set
    Y_pred = pipeline.predict(X_test)

    rmse = np.sqrt(mean_squared_error(Y_test, Y_pred))
    mae = mean_absolute_error(Y_test, Y_pred)
    r2 = r2_score(Y_test, Y_pred)

    # Calculate the metrics
    results[name] = {
        'CV RMSE': rmse_scores.mean(),
        'Test RMSE': rmse,
        'Test MAE': mae,
        'Test R2': r2,
        'Model': pipeline
    }

    print(f'{name} done')
    print(f'CV RMSE: {rmse_scores.mean()}')
    print(f'Test RMSE: {rmse}')
    print(f'Test MAE: {mae}')
    print(f'Test R2: {r2}')
    print('\n')


# # Training and Evaluation
# for name, model in models.items():
#     # Create pipeline
#     pipeline = Pipeline(steps=[('preprocessor', preprocessor), ('model', model)])

#     # Cross-validation
#     cv_scores = cross_val_score(pipeline, X_train, Y_train, cv=5, scoring='neg_mean_squared_error')
#     # Calculate RMSE
#     rmse_scores = np.sqrt(-cv_scores)

#     # Train on full training set
#     pipeline.fit(X_train, Y_train)

    # # Predict on test set
    # Y_pred = pipeline.predict(X_test)

    # # Calculate metrics
    # rmse = np.sqrt(mean_squared_error(Y_test, Y_pred))
    # mae = mean_absolute_error(Y_test, Y_pred)
    # r2 = r2_score(Y_test, Y_pred)

    # results[name] = {
    #     'CV RMSE': rmse_scores.mean(),
    #     'Test RMSE': rmse,
    #     'Test MAE': mae,
    #     'Test R2': r2,
    #     'Model': pipeline
    # }

    # print(f'{name} done')
    # print(f'CV RMSE: {rmse_scores.mean()}')
    # print(f'Test RMSE: {rmse}')
    # print(f'Test MAE: {mae}')
    # print(f'Test R2: {r2}')
    # print('\n')

Linear Regression done
CV RMSE: 2663.1317276754553
Test RMSE: 2711.8729809968445
Test MAE: 1811.2471238530318
Test R2: 0.6589260804026489


Random Forest done
CV RMSE: 1944.1533394161754
Test RMSE: 1983.562466795263
Test MAE: 881.2571593243755
Test R2: 0.8175257498390817


Gradient Boosting done
CV RMSE: 2234.647971844866
Test RMSE: 2225.7343430786914
Test MAE: 1411.5851359683213
Test R2: 0.770249493300472


XGBoost done
CV RMSE: 1864.7659060413953
Test RMSE: 1802.7160758144917
Test MAE: 964.3184204101562
Test R2: 0.8492822051048279




/Users/shifaafreensiddiqui/Library/Python/3.9/lib/python/site-packages/sklearn/neural_network/_multilayer_perceptron.py:691: ConvergenceWarning: Stochastic Optimizer: Maximum iterations (200) reached and the optimization hasn't converged yet.
  warnings.warn(
/Users/shifaafreensiddiqui/Library/Python/3.9/lib/python/site-packages/sklearn/neural_network/_multilayer_perceptron.py:691: ConvergenceWarning: Stochastic Optimizer: Maximum iterations (200) reached and the optimization hasn't converged yet.
  warnings.warn(
/Users/shifaafreensiddiqui/Library/Python/3.9/lib/python/site-packages/sklearn/neural_network/_multilayer_perceptron.py:691: ConvergenceWarning: Stochastic Optimizer: Maximum iterations (200) reached and the optimization hasn't converged yet.
  warnings.warn(
/Users/shifaafreensiddiqui/Library/Python/3.9/lib/python/site-packages/sklearn/neural_network/_multilayer_perceptron.py:691: ConvergenceWarning: Stochastic Optimizer: Maximum iterations (200) reached and the optimization

MLP done
CV RMSE: 3112.8948163322466
Test RMSE: 2914.7467848332844
Test MAE: 2044.6718179696168
Test R2: 0.6059861503016346




/Users/shifaafreensiddiqui/Library/Python/3.9/lib/python/site-packages/sklearn/neural_network/_multilayer_perceptron.py:691: ConvergenceWarning: Stochastic Optimizer: Maximum iterations (200) reached and the optimization hasn't converged yet.
  warnings.warn(


In [27]:
# Find the best model
best_model_name = min(results, key=lambda x: results[x]['Test RMSE'])
best_model = results[best_model_name]['Model']

print(f"\nBest model: {best_model_name}")
print(f"Test RMSE: {results[best_model_name]['Test RMSE']:.2f}")
print(f"Test MAE: {results[best_model_name]['Test MAE']:.2f}")
print(f"Test R2: {results[best_model_name]['Test R2']:.2f}")


Best model: XGBoost
Test RMSE: 1802.72
Test MAE: 964.32
Test R2: 0.85


In [28]:
import matplotlib.pyplot as plt
import seaborn as sns

# Feature importance (for tree-based models)
if best_model_name in ['Random Forest', 'Gradient Boosting']:
    # Get feature names after preprocessing
    cat_feature_names = preprocessor.transformers_[1][1].get_feature_names_out(categorical_features)
    feature_names = numerical_features + list(cat_feature_names)
    
    # Get feature importances
    importances = best_model.named_steps['model'].feature_importances_
    
    # Create a DataFrame for visualization
    feature_importance = pd.DataFrame({
        'Feature': feature_names,
        'Importance': importances
    })
    
    # Sort by importance
    feature_importance = feature_importance.sort_values('Importance', ascending=False)
    
    # Plot feature importance
    plt.figure(figsize=(10, 6))
    sns.barplot(x='Importance', y='Feature', data=feature_importance)
    plt.title('Top 15 Feature Importances')
    plt.tight_layout()
    plt.show()

In [29]:
# Predict on test set
test = pd.DataFrame()

test['date_of_journey'] = pd.to_datetime(test_raw['Date_of_Journey'], dayfirst=True)
test['day_of_week'] = test['date_of_journey'].dt.dayofweek
test['month'] = test['date_of_journey'].dt.month
test['year'] = test['date_of_journey'].dt.year

test['total_stops'] = test_raw['Total_Stops'].apply(extract_stops)

test['dep_hour'] = test_raw['Dep_Time'].apply(extract_hour)
test['arr_hour'] = test_raw['Arrival_Time'].apply(extract_hour)

test['duration'] = test_raw['Duration'].apply(duration_to_minutes)

test['source'] = test_raw['Route'].apply(lambda x: x.split('→')[0].strip())
test['destination'] = test_raw['Route'].apply(lambda x: x.split('→')[-1].strip())

test['route_complexity'] = test_raw['Route'].apply(lambda x: len(x.split('→')) - 2)

additional_info_df = test_raw['Additional_Info'].apply(process_additional_info)
test = pd.concat([test, additional_info_df], axis=1)

test['airline'] = test_raw['Airline']

# Predict
Y_pred = best_model.predict(test)

# Add predictions to test set
test['price'] = Y_pred

test.head()

,date_of_journey,day_of_week,month,year,total_stops,dep_hour,arr_hour,duration,source,destination,route_complexity,has_meal,check_in_baggage,short_layover,long_layover,red_eye,business_class,change_airport,airline,price
0,2019-06-06,3,6,2019,1,18,4,655,DEL,COK,1,0,0,0,0,0,0,0,Jet Airways,14542.256836
1,2019-05-12,6,5,2019,1,6,10,240,CCU,BLR,1,0,0,0,0,0,0,0,IndiGo,5162.723145
2,2019-05-21,1,5,2019,1,19,19,1425,DEL,COK,1,1,0,0,0,0,0,0,Jet Airways,12590.912109
3,2019-05-21,1,5,2019,1,8,21,780,DEL,COK,1,0,0,0,0,0,0,0,Multiple carriers,10741.335938
4,2019-06-24,0,6,2019,0,24,3,170,BLR,DEL,0,0,0,0,0,0,0,0,Air Asia,5695.134277
